# Objetivo

Este script es el tercero de una serie de scripts utilizados en el TFM para crear un corpus adicional de notas con el que enriquecer el conjunto de entrenamiento de MEDDOPROF.

Como input lee las notas almacenadas en la carpeta `/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas`. Provenientes del script `2ExtraccionNotas`

Una vez leídas estas notas vemos si existen duplicados entre ellas. Esto es totalmente plausible puesto que estamos cogiendo notas de varios corpus para construir el nuevo conjunto de datos. En esos corpus la misma nota puede tener diferentes nombres pero el contenido puede ser el mismo.

Este script se divide en dos partes fundamentales:

1. Comprobación de notas duplicadas mediante la función `duplicated`
2. Comprobación de notas duplicadas muy parecidas pero que no sean exactas (se da el caso de que hay notas duplicados con distinto nivel de indexación o a las que se ha eliminado la cabecera en un corpus y en otro no)

El output de este script es un mensaje en pantalla que nos indica:

1. Notas duplicadas
2. Semejanza entre pares de notas mediante TF-IDF. El valor obtenido lo vamos a usar para revisar finalmente las notas por si son duplicadas.


# Duplicidades sencillas

In [ ]:
# Cargamos las librerías necesarias
import numpy as np
import pandas as pd
import re
import os, glob
from sklearn.datasets import load_files
import shutil

# Habilitamos display interactivo y enlace con nuestra cuenta de google
from google.colab import data_table
from google.colab import drive

# Enlazamos nuestro notebook en Colab con nuestro almacenamiento en Google Drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define relative path to folder containing the text files
files_folder = "/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas"
notas = []

# Create a dataframe list by using a list comprehension
notas = [pd.read_csv(file, delimiter='\t') for file in glob.glob(os.path.join(files_folder ,"*.txt"))]
notas = pd.DataFrame(notas, columns = ['notas'])

/usr/local/lib/python3.8/dist-packages/pandas/core/internals/construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [ ]:
# Sacamos la ruta de cada archivo
glob.glob(os.path.join(files_folder ,"*.txt"))

['/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/es-S1137-66272013000200022-1.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/es-S0376-78922009000100011-1.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/es-S1137-66272011000100013-1.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/caso_clinico_urologia434.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/caso_clinico_atencion_primaria20.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/casos_clinicos_cardiologia268.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/casos_clinicos_cardiologia358.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/casos_clinicos_cardiologia447.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/casos_clinicos_cardiologia351.txt',
 '/content/drive/MyDrive/Colab Notebooks/anotar/NotasDefinitivas/32617223_ES.txt',
 '/content/dr

In [ ]:
# Eliminamos la ruta completa y dejamos solo el nombre del archivo
df = pd.DataFrame(notas, columns = ['notas'])
df['notas']=df['notas'].apply(str)
df["nombre"] = glob.glob(os.path.join(files_folder ,"*.txt"))
df['nombre'] = df.nombre.apply(lambda x: os.path.splitext(os.path.basename(x))[0])

In [ ]:
#Vemos el df
df

,notas,nombre
0,Paciente de 33 años de edad sin antecedentes...,es-S1137-66272013000200022-1
1,"Varón soltero de 37 años de edad, trabajado...",es-S0376-78922009000100011-1
2,"Empty DataFrame\nColumns: [Varón de 45 años, t...",es-S1137-66272011000100013-1
3,Anámnesis e historia ...,caso_clinico_urologia434
4,Motivo de c...,caso_clinico_atencion_primaria20
...,...,...
472,Paciente de 18 años de edad y sexo femenino ...,S0376-78922009000200008-1
473,Mujer de 47 años que consulta en urgencias p...,S1887-85712012000400006-1
474,Paciente de 33 años de edad sin antecedentes...,S1137-66272013000200022-1
475,Mujer de 40 años que acude a la consulta de ...,S1699-695X2015000200010-1


Definimos una pipeline de pre-procesado para encontrar duplicidades sencillas

In [ ]:
import nltk
from nltk.stem import *
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
# import contractions
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4') # Lo añadimos nuevo

# Cargamos el stemmer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Eliminamos stopwords y nos quedamos únicamente con caracteres alfanuméricos

In [ ]:
def process_text(raw_text):
    cleantext = re.sub(re.compile('<.*?>'), '', raw_text)
    #Consideramos únicamente letras utilizando una expresión regular
    letters_only = re.sub("[^a-zA-Z]", " ", cleantext) 
    #Convertimos todo a minúsculas (normalizamos)
    words = letters_only.lower().split()
    
    #Eliminamos las stopwords
    stops = set(stopwords.words("spanish")) 
    not_stop_words = [w for w in words if not w in stops]
  
    #Stemming
    stemmer = SnowballStemmer('spanish')
    stemmed = [stemmer.stem(word) for word in not_stop_words]
    
    return(" ".join(stemmed))  

In [ ]:
# Creamos una columna con la nota limpia
df['notas_limpias'] = df['notas'].apply(lambda x: process_text(x))

In [ ]:
df

,notas,nombre,notas_limpias
0,Paciente de 33 años de edad sin antecedentes...,es-S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...
1,"Varón soltero de 37 años de edad, trabajado...",es-S0376-78922009000100011-1,var n solter edad trabaj camp envi servici hos...
2,"Empty DataFrame\nColumns: [Varón de 45 años, t...",es-S1137-66272011000100013-1,empty datafram columns var n trabaj canter dia...
3,Anámnesis e historia ...,caso_clinico_urologia434,an mnesis histori cl nic acud deriv m dic aten...
4,Motivo de c...,caso_clinico_atencion_primaria20,motiv consult voluntad vital anticip vva enfoq...
...,...,...,...
472,Paciente de 18 años de edad y sexo femenino ...,S0376-78922009000200008-1,pacient edad sex femenin oper extrofi cloac re...
473,Mujer de 47 años que consulta en urgencias p...,S1887-85712012000400006-1,muj consult urgenci cuadr hor evoluci n dolor ...
474,Paciente de 33 años de edad sin antecedentes...,S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...
475,Mujer de 40 años que acude a la consulta de ...,S1699-695X2015000200010-1,muj acud consult atenci n primari dolor ambas ...


Duplicados conjunto a anotar, con el que vamos a enriquecer (MOD)

In [ ]:
# Señalamos los duplicados
df["duplicados"] = df.notas_limpias.duplicated(keep = False)
df[df.duplicados == True].sort_values("notas")

,notas,nombre,notas_limpias,duplicados
224,Niño de 9 años con faringitis desde hace 7 d...,es-S0210-56912007000200007-3,faringitis hac d as fiebr elev artralgi hor pr...,True
451,Niño de 9 años con faringitis desde hace 7 d...,S0210-56912007000200007-3,faringitis hac d as fiebr elev artralgi hor pr...,True
0,Paciente de 33 años de edad sin antecedentes...,es-S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True
474,Paciente de 33 años de edad sin antecedentes...,S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True
225,"Paciente de 42 años, casada, unigesta (gemel...",es-S0376-78922009000300005-1,pacient cas unigest gemel histori infeccion re...,True
469,"Paciente de 42 años, casada, unigesta (gemel...",S0376-78922009000300005-1,pacient cas unigest gemel histori infeccion re...,True
223,Se trata de un varón de 66 años con antecede...,es-S0210-56912009000900008-1,trat var n antecedent neoplasi pulmon carcinom...,True
452,Se trata de un varón de 66 años con antecede...,S0210-56912009000900008-1,trat var n antecedent neoplasi pulmon carcinom...,True
297,Varón de 27 años que presenta fiebre y hemop...,caso_clinico_radiologia867,var n present fiebr hemoptisis antecedent pers...,True
396,Varón de 27 años que presenta fiebre y hemop...,S1137-66272006000100012-1,var n present fiebr hemoptisis antecedent pers...,True


Con el paso anterior localizamos notas con distinto nombre y mismo contenido entre las notas de los seis corpus que estamos utilizando para enriquecer el conjunto de MEDDOPROF (el que será el conjunto MOD)

A continuación queremos identificar si hay casos en los que las notas seleccionadas para enriquecer MEDDOPROF también formen parte de este último bajo un nombre distinto (tanto el conjunto de entrenamiento como el conjunto de test). Esto es relevante dado que podríamos entrenar con una nota aparentemente nueva (localizada en alguno de los seis corpus) que estuviera en el conjunto de test de MEDDOPROF


Cargamos el conjunto de datos de train de MEDDOPROF

In [ ]:
DATA_DIR_TRAIN = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Original/Train/"
data = load_files(DATA_DIR_TRAIN, encoding="utf-8", decode_error="replace", shuffle=False)
data_train = pd.DataFrame(data.data, columns = ['texto'])
data_train['fichero'] = data.filenames

# Nos quedamos únicamente con los ficheros de estudio en base a como se organiza el corpus en carpetas
data_train = data_train[(data_train.fichero.str.contains('.txt', case=False, regex=False)) & (data_train.fichero.str.contains('task1', case=False, regex=False))]

#df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)

#Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
data_train['fichero'] = data_train["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
data_train.head(5)

,texto,fichero
1,Un paciente de unos cincuenta años sin anteced...,32247016_ES
3,"﻿Una mujer de 27 años sin antecedentes, reside...",32294273_ES
5,﻿Mujer de 66 años que acudió refiriendo dificu...,32399950_ES
7,﻿El 28 de febrero de 2020 ingresó en urgencias...,32417987_ES
9,"﻿Una mujer de 31 años, por lo demás sana, acud...",32426200_ES


In [ ]:
# Procesamos los textos del conjunto de train tal y como hicimos anteriormente
data_train['notas_limpias'] = data_train['texto'].apply(lambda x: process_text(x))

In [ ]:
data_train

,texto,fichero,notas_limpias
1,Un paciente de unos cincuenta años sin anteced...,32247016_ES,pacient cincuent antecedent personal acudi m d...
3,"﻿Una mujer de 27 años sin antecedentes, reside...",32294273_ES,muj antecedent resident hospital present odino...
5,﻿Mujer de 66 años que acudió refiriendo dificu...,32399950_ES,muj acudi refir dificult andar fatig agud hor ...
7,﻿El 28 de febrero de 2020 ingresó en urgencias...,32417987_ES,febrer ingres urgenci bomber tras crisis carac...
9,"﻿Una mujer de 31 años, por lo demás sana, acud...",32426200_ES,muj dem s san acudi servici urgenci s ntom fie...
...,...,...,...
2991,Se presenta el caso de una paciente de 28 años...,cc_otorrinolaringologia54,present cas pacient present hipoacusi hor evol...
2993,Paciente mujer de 81 años con hipoacusia bilat...,cc_otorrinolaringologia56,pacient muj hipoacusi bilateral progres larg e...
2995,Se presenta el caso de una mujer de 33 años qu...,cc_otorrinolaringologia60,present cas muj acud urgenci odinofagi fiebr e...
2997,Se presenta el caso de un varón de 18 años de ...,cc_otorrinolaringologia72,present cas var n edad jornaler profesi n pres...


Vemos si existen duplicados en el conjunto de ENTRENAMIENTO de MEDDOPROF únicamente

In [ ]:
# Señalamos los duplicados en el conjunto de entrenamiento
data_train["duplicados"] = data_train.notas_limpias.duplicated(keep = False)
data_train[data_train.duplicados == True].sort_values("texto")

,texto,fichero,notas_limpias,duplicados
257,"Motivo de Consulta\nAgresividad, impulsividad ...",caso_clinico_atencion_primaria161,motiv consult agres impuls insomni mal control...,True
259,"Motivo de Consulta\nAgresividad, impulsividad ...",caso_clinico_atencion_primaria162,motiv consult agres impuls insomni mal control...,True


Conjunto de test

In [ ]:
DATA_DIR_TEST = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Original/Test/"
data = load_files(DATA_DIR_TEST, encoding="utf-8", decode_error="replace", shuffle=False)

data_test = pd.DataFrame(data.data, columns = ['texto'])
data_test['fichero'] = data.filenames

# Nos quedamos únicamente con los ficheros de estudio en base a como se organiza el corpus en carpetas
data_test = data_test[(data_test.fichero.str.contains('.txt', case=False, regex=False)) & (data_test.fichero.str.contains('ner', case=False, regex=False))]

#df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)

#Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
data_test['fichero'] = data_test["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
data_test.head(5)

,texto,fichero
689,﻿Un hombre de 36 años llegó al servicio de urg...,32423911_ES
691,"﻿El señor G, un hombre de 43 años sin antecede...",32605766_ES
693,﻿Una mujer de 20 años embarazada secundigesta ...,32614251_ES
695,Nombre: Miriam .\nApellidos: Aguilar Mora.\nC...,S0004-06142009000100010-2
697,Nombre: Paula.\nApellidos: De la Cruz Lorenzo....,S0034-98872006000200011-1


In [ ]:
# Procesamos los textos del conjunto de train tal y como hicimos anteriormente
data_test['notas_limpias'] = data_test['texto'].apply(lambda x: process_text(x))

Vemos si existen duplicados en el conjunto de TEST de MEDDOPROF únicamente

In [ ]:
# Señalamos los duplicados en el conjunto de test
data_test["duplicados"] = data_test.notas_limpias.duplicated(keep = False)
data_test[data_test.duplicados == True].sort_values("texto")

,texto,fichero,notas_limpias,duplicados
1051,"Hombre de 47 años, empleado de la industria me...",casos_clinicos_profesiones120,hombr emple industri metal rgic oper maquinari...,True
1199,"Hombre de 47 años, empleado de la industria me...",casos_clinicos_profesiones193,hombr emple industri metal rgic oper maquinari...,True


Combinamos los distintos archivos para encontrar duplicados entre el conjunto de datos con el que vamos a enriquecer el entrenamiento con los datos de MEDDOPROF

In [ ]:
# Creamos tres objetos que tengan el mismo aspecto para concatenar en un solo dataframe las notas provenientes de MEDDOPROF TRAIN/TEST con el conjunto de datos usado para enriquecer
# Conjunto de train procesado
df_train_procesado = data_train[["fichero", "notas_limpias", "duplicados"]]
df_train_procesado["conjunto"] = "train"
#df_train_procesado

# Conjunto de test procesado
df_test_procesado = data_test[["fichero", "notas_limpias", "duplicados"]]
df_test_procesado["conjunto"] = "test"
#df_test_procesado

# Conjunto de anotación con el que vamos a enriquecer
df_anotacion = df[["nombre", "notas_limpias", "duplicados"]].rename(columns = {'nombre':'fichero'})
df_anotacion["conjunto"] = "anotar"

In [ ]:
df_final = pd.concat([df_train_procesado, df_test_procesado, df_anotacion]).reset_index()
df_final

,index,fichero,notas_limpias,duplicados,conjunto
0,1,32247016_ES,pacient cincuent antecedent personal acudi m d...,False,train
1,3,32294273_ES,muj antecedent resident hospital present odino...,False,train
2,5,32399950_ES,muj acudi refir dificult andar fatig agud hor ...,False,train
3,7,32417987_ES,febrer ingres urgenci bomber tras crisis carac...,False,train
4,9,32426200_ES,muj dem s san acudi servici urgenci s ntom fie...,False,train
...,...,...,...,...,...
2316,472,S0376-78922009000200008-1,pacient edad sex femenin oper extrofi cloac re...,False,anotar
2317,473,S1887-85712012000400006-1,muj consult urgenci cuadr hor evoluci n dolor ...,False,anotar
2318,474,S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True,anotar
2319,475,S1699-695X2015000200010-1,muj acud consult atenci n primari dolor ambas ...,False,anotar


Vemos todos los duplicados (en el conjunto MOD, test y train) sin aplicar TF-IDF, observando a qué conjunto pertenecen. Vemos que no hay notas cruzadas es decir, todas las notas duplicadas pertenecen al mismo conjunto

In [ ]:
# Señalamos los duplicados de la combinación de archivos
df_final["duplicados"] = df_final.notas_limpias.duplicated(keep = False)
df_final[df_final.duplicados == True].sort_values("notas_limpias")

,index,fichero,notas_limpias,duplicados,conjunto
2068,224,es-S0210-56912007000200007-3,faringitis hac d as fiebr elev artralgi hor pr...,True,anotar
2295,451,S0210-56912007000200007-3,faringitis hac d as fiebr elev artralgi hor pr...,True,anotar
1681,1051,casos_clinicos_profesiones120,hombr emple industri metal rgic oper maquinari...,True,test
1755,1199,casos_clinicos_profesiones193,hombr emple industri metal rgic oper maquinari...,True,test
128,257,caso_clinico_atencion_primaria161,motiv consult agres impuls insomni mal control...,True,train
129,259,caso_clinico_atencion_primaria162,motiv consult agres impuls insomni mal control...,True,train
2069,225,es-S0376-78922009000300005-1,pacient cas unigest gemel histori infeccion re...,True,anotar
2313,469,S0376-78922009000300005-1,pacient cas unigest gemel histori infeccion re...,True,anotar
1844,0,es-S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True,anotar
2318,474,S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True,anotar


In [ ]:
# Vemos el aspecto del conjunto de datos de las notas encontradas
df

,notas,nombre,notas_limpias,duplicados
0,Paciente de 33 años de edad sin antecedentes...,es-S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True
1,"Varón soltero de 37 años de edad, trabajado...",es-S0376-78922009000100011-1,var n solter edad trabaj camp envi servici hos...,False
2,"Empty DataFrame\nColumns: [Varón de 45 años, t...",es-S1137-66272011000100013-1,empty datafram columns var n trabaj canter dia...,False
3,Anámnesis e historia ...,caso_clinico_urologia434,an mnesis histori cl nic acud deriv m dic aten...,False
4,Motivo de c...,caso_clinico_atencion_primaria20,motiv consult voluntad vital anticip vva enfoq...,False
...,...,...,...,...
472,Paciente de 18 años de edad y sexo femenino ...,S0376-78922009000200008-1,pacient edad sex femenin oper extrofi cloac re...,False
473,Mujer de 47 años que consulta en urgencias p...,S1887-85712012000400006-1,muj consult urgenci cuadr hor evoluci n dolor ...,False
474,Paciente de 33 años de edad sin antecedentes...,S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True
475,Mujer de 40 años que acude a la consulta de ...,S1699-695X2015000200010-1,muj acud consult atenci n primari dolor ambas ...,False


# Duplicidades TF-IDF

Implementación del enfoque TF-IDF para calcular la similitud entre documentos. Cuanto más cerca el valor a 1, más parecidos son.

## Duplicidades notas del conjunto MOD

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer().fit_transform(df.notas_limpias)
pairwise_similarity = tfidf * tfidf.T
# Calculamos la similitud
conjunto_MOD = pairwise_similarity.toarray()
conjunto_MOD

array([[1.        , 0.01788417, 0.0295041 , ..., 1.        , 0.01977448,
        0.08711051],
       [0.01788417, 1.        , 0.05835762, ..., 0.01788417, 0.04574716,
        0.07048654],
       [0.0295041 , 0.05835762, 1.        , ..., 0.0295041 , 0.0355382 ,
        0.10160884],
       ...,
       [1.        , 0.01788417, 0.0295041 , ..., 1.        , 0.01977448,
        0.08711051],
       [0.01977448, 0.04574716, 0.0355382 , ..., 0.01977448, 1.        ,
        0.02120958],
       [0.08711051, 0.07048654, 0.10160884, ..., 0.08711051, 0.02120958,
        1.        ]])

Transformamos la matriz de similitud a formato long para analizar qué pares de notas son las que tienen mayor similitud. Esto sólo lo hacemos, en el código de abajo, para las notas usasdas para enriquecer el conjunto de datos. Se compara una la similitud de una nota con todas las demás (477 * 477)

In [ ]:
conjunto_MOD = pd.DataFrame(conjunto_MOD)
# Aplanamos la matriz para trabajar
conjunto_MOD = conjunto_MOD.unstack()
conjunto_MOD = conjunto_MOD.reset_index()
conjunto_MOD

,level_0,level_1,0
0,0,0,1.000000
1,0,1,0.017884
2,0,2,0.029504
3,0,3,0.035997
4,0,4,0.005037
...,...,...,...
227524,476,472,0.005852
227525,476,473,0.076561
227526,476,474,0.087111
227527,476,475,0.021210


In [ ]:
# Creamos el conjunto de datos. Las dos primeras columnas identifican la posición original en la matriz del valor TF-IDF
conjunto_MOD = conjunto_MOD.set_axis(['row', 'column', 'value'], axis=1, inplace=False)
conjunto_MOD = conjunto_MOD.sort_values(by=["value"], ascending = False)
conjunto_MOD = conjunto_MOD[conjunto_MOD.row != conjunto_MOD.column]
# Ponemos punto de corte a 0.35 de forma arbritaria
conjunto_MOD = conjunto_MOD[conjunto_MOD.value > 0.35]
conjunto_MOD

,row,column,value
107299,224,451,1.000000
215351,451,224,1.000000
223938,469,225,1.000000
107794,225,469,1.000000
189189,396,297,1.000000
142065,297,396,1.000000
474,0,474,1.000000
226098,474,0,1.000000
106823,223,452,1.000000
215827,452,223,1.000000


Vemos que hay 26 notas, en el conjunto MOD, con un valor de TF-IDF > 0.35

In [ ]:
len(conjunto_MOD)

52

Imprimimos qué notas son

In [ ]:
# Mediante un bucle recorremos todos los archivos con alta similiradidad pareados dos a dos
for i in range(0, len(conjunto_MOD)):
  print("Notas repetidas:", df.iloc[conjunto_MOD.iloc[i,0],1], df.iloc[conjunto_MOD.iloc[i,1],1])

Notas repetidas: es-S0210-56912007000200007-3 S0210-56912007000200007-3
Notas repetidas: S0210-56912007000200007-3 es-S0210-56912007000200007-3
Notas repetidas: S0376-78922009000300005-1 es-S0376-78922009000300005-1
Notas repetidas: es-S0376-78922009000300005-1 S0376-78922009000300005-1
Notas repetidas: S1137-66272006000100012-1 caso_clinico_radiologia867
Notas repetidas: caso_clinico_radiologia867 S1137-66272006000100012-1
Notas repetidas: es-S1137-66272013000200022-1 S1137-66272013000200022-1
Notas repetidas: S1137-66272013000200022-1 es-S1137-66272013000200022-1
Notas repetidas: es-S0210-56912009000900008-1 S0210-56912009000900008-1
Notas repetidas: S0210-56912009000900008-1 es-S0210-56912009000900008-1
Notas repetidas: cc_covid81 cc_covid114
Notas repetidas: cc_covid114 cc_covid81
Notas repetidas: casos_clinicos_cardiologia470 casos_clinicos_cardiologia44
Notas repetidas: casos_clinicos_cardiologia44 casos_clinicos_cardiologia470
Notas repetidas: cc_reumatologia238 cc_reumatologia2



---



## Duplicidades notas de todos los conjuntos a la vez

Comprobamos la no pertenencia de notas del conjunto de datos con el que vamos a enriquecer (MOD), en train y test.

Hacemos un análisis TF-IDF con los tres conjuntos de datos (train, test y las notas identificadas) a la vez

In [ ]:
# Ordenamos para que notas pertenecientes a conjuntos en concreto estén consecutivas
df_final = df_final.sort_values(["conjunto", "index"])
df_final

,index,fichero,notas_limpias,duplicados,conjunto
1844,0,es-S1137-66272013000200022-1,pacient edad antecedent inter s diagnostic enf...,True,anotar
1845,1,es-S0376-78922009000100011-1,var n solter edad trabaj camp envi servici hos...,False,anotar
1846,2,es-S1137-66272011000100013-1,empty datafram columns var n trabaj canter dia...,False,anotar
1847,3,caso_clinico_urologia434,an mnesis histori cl nic acud deriv m dic aten...,False,anotar
1848,4,caso_clinico_atencion_primaria20,motiv consult voluntad vital anticip vva enfoq...,False,anotar
...,...,...,...,...,...
1495,2991,cc_otorrinolaringologia54,present cas pacient present hipoacusi hor evol...,False,train
1496,2993,cc_otorrinolaringologia56,pacient muj hipoacusi bilateral progres larg e...,False,train
1497,2995,cc_otorrinolaringologia60,present cas muj acud urgenci odinofagi fiebr e...,False,train
1498,2997,cc_otorrinolaringologia72,present cas var n edad jornaler profesi n pres...,False,train


In [ ]:
df_final.conjunto.value_counts()

train     1500
anotar     477
test       344
Name: conjunto, dtype: int64

Las observaciones que van desde la 0 a la 477 se corresponden con el conjunto a anotar

In [ ]:
df_final.iloc[476]

index                                                          476
fichero                                  S0211-69952011000500011-3
notas_limpias    empty datafram columns hombr diabet tip larg e...
duplicados                                                   False
conjunto                                                    anotar
Name: 2320, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer().fit_transform(df_final.notas_limpias)
pairwise_similarity = tfidf * tfidf.T
procesado_todosConjuntos = pairwise_similarity.toarray()
procesado_todosConjuntos

array([[1.        , 0.01452653, 0.02689272, ..., 0.0190059 , 0.02443002,
        0.03169734],
       [0.01452653, 1.        , 0.04839181, ..., 0.04549546, 0.11495877,
        0.04581697],
       [0.02689272, 0.04839181, 1.        , ..., 0.05943574, 0.07723135,
        0.0499771 ],
       ...,
       [0.0190059 , 0.04549546, 0.05943574, ..., 1.        , 0.03222219,
        0.05447175],
       [0.02443002, 0.11495877, 0.07723135, ..., 0.03222219, 1.        ,
        0.07594992],
       [0.03169734, 0.04581697, 0.0499771 , ..., 0.05447175, 0.07594992,
        1.        ]])

In [ ]:
procesado_todosConjuntos = pd.DataFrame(procesado_todosConjuntos)
# Aplanamos la matriz para trabajar
procesado_todosConjuntos = procesado_todosConjuntos.unstack()
procesado_todosConjuntos = procesado_todosConjuntos.reset_index()
procesado_todosConjuntos

,level_0,level_1,0
0,0,0,1.000000
1,0,1,0.014527
2,0,2,0.026893
3,0,3,0.025827
4,0,4,0.005028
...,...,...,...
5387036,2320,2316,0.038202
5387037,2320,2317,0.012550
5387038,2320,2318,0.054472
5387039,2320,2319,0.075950


Relaciones con un valor de TF-IDF mayor a 0.35

In [ ]:
# Creamos el conjunto de datos. Las dos primeras columnas identifican la posición original en la matriz del valor TF-IDF
procesado_todosConjuntos = procesado_todosConjuntos.set_axis(['row', 'column', 'value'], axis=1, inplace=False)
procesado_todosConjuntos = procesado_todosConjuntos.sort_values(by=["value", 'row', 'column'], ascending = False)
# Eliminamos aquellas observaciones de notas que son similares consigo mismas
procesado_todosConjuntos = procesado_todosConjuntos[procesado_todosConjuntos.row != procesado_todosConjuntos.column]

# Ponemos punto de corte a 0.35 de forma arbritaria
procesado_todosConjuntos = procesado_todosConjuntos[procesado_todosConjuntos.value > 0.35]
procesado_todosConjuntos

,row,column,value
1100154,474,0,1.000000
1088774,469,225,1.000000
919413,396,297,1.000000
689733,297,396,1.000000
522694,225,469,1.000000
...,...,...,...
1385047,596,1731,0.350206
4227144,1821,603,0.350122
1401384,603,1821,0.350122
4620750,1990,1960,0.350079


In [ ]:
# Número de relaciones con más de un 0.4 de valor TF-IDF
len(procesado_todosConjuntos)

1578

Vemos las notas con dicho valor de relación

In [ ]:
# Mediante un bucle recorremos todos los archivos con alta similiradidad pareados dos a dos
#for i in range(0, len(procesado_todosConjuntos)):
#  print("Notas repetidas:", df_final.iloc[procesado_todosConjuntos.iloc[i,0],1], df_final.iloc[procesado_todosConjuntos.iloc[i,1],1])



---



Vamos a comparar las notas a anotar (MOD) con las notas del conjunto de train, por un lado y con las notas del conjunto de test por otro lado)

Lo hacemos así porque no tiene sentido comparar la similitud entre las notas del conjunto MEDDOPROF train con las notas del conjunto MEDDOPROF test, o las notas del conjunto MOD consigo mismas.

Dado que `procesado_todosConjuntos` viene de `datos_final` que fue ordenado por la columna conjunto, las 477 primeras observaciones vienen del conjunto MOD (por tanto la columa row o column debe tener valores de este conjunto y la columna complementaria column o row debe tener valores fuera de este conjunto). Así evitamos volver a comparar las notas que provienen del conjunto MOD dos a dos.

In [ ]:
#De los potenciales conflictos nos quedamos con aquellos cuya fila es menor de 477 (corresponden con las notas de train) y una columna mayor o igual a 477, es decir que caiga en train o en test)
similaritud_anotar = procesado_todosConjuntos[(procesado_todosConjuntos.row < 477) & (procesado_todosConjuntos.column >= 477)]
similaritud_anotar

,row,column,value
891934,384,670,0.981696
159885,68,2057,0.944289
246843,106,817,0.919287
166805,71,2014,0.853177
5516,2,874,0.785477
144777,62,875,0.785043
251544,108,876,0.731334
881242,379,1583,0.673338
328118,141,857,0.673209
58843,25,818,0.661398


In [ ]:
len(similaritud_anotar)

28

In [ ]:
# Mediante un bucle recorremos todos los archivos con alta similaridad pareados dos a dos
for i in range(0, len(similaritud_anotar)):
  print("Notas repetidas:", df_final.iloc[similaritud_anotar.iloc[i,0],1], df_final.iloc[similaritud_anotar.iloc[i,1],1])

Notas repetidas: S0365-66912011001000003-4 casos_clinicos_profesiones132
Notas repetidas: cc_reumatologia353 casos_clinicos_profesiones79
Notas repetidas: cc_reumatologia60 cc_reuma56
Notas repetidas: es-S0465-546X2014000400012-1 casos_clinicos_profesiones3
Notas repetidas: es-S1137-66272011000100013-1 S1137-66272011000100013-1
Notas repetidas: es-S1137-66272011000100013-2 S1137-66272011000100013-2
Notas repetidas: es-S1137-66272011000100013-3 S1137-66272011000100013-3
Notas repetidas: S0211-57352014000400011-1 caso_clinico_psiquiatria306
Notas repetidas: es-S0465-546X2014000300010-1 S0465-546X2014000300010-1
Notas repetidas: cc_reumatologia62 cc_reuma58
Notas repetidas: es-S0465-546X2014000400012-1 S0465-546X2014000400012-1
Notas repetidas: caso_clinico_oftalmologia285 caso_clinico_oftalmologia40
Notas repetidas: S0211-57352014000400010-1 caso_clinico_psiquiatria305
Notas repetidas: cc_geneticas200 caso_clinico_psiquiatria372
Notas repetidas: es-S0465-546X2009000300008-1 S0465-546X200

Después de aplicar este script se deben anotar las notas con BRAT